In [3]:
from PyPDF2 import PdfReader
import os
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chat_models import ChatOpenAI

In [2]:
def extract_text_from_pdf(url: str) -> str:
    reader = PdfReader(url)

    text = ''
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()
    return text

In [5]:
base_path = '../../data/raw/dataset/'
content = []
instruction = []
output = []
with os.scandir(base_path) as entries:
    for entry in entries:
        if entry.name.endswith(".pdf"):
            text = extract_text_from_pdf(base_path+entry.name)
            components = text.split(":")

            name = components[1].replace('Name', '').replace('Description','')#Name and Description
            description = components[2].replace('Data', '')# Description and Data
            data = text.split(":")[3].replace('In-depth Analysis', '').replace('-', '')
            analysis = text.split(":")[4] # In-depth Analysis
            text = name + ' '+ description + ' '+ data + ' '+analysis
            content.append(text.replace('\n', ''))
        elif entry.name.endswith(".txt"):
            id = lambda x: x
            file = open(base_path+entry.name,"r+", encoding="utf8")
            lines = file.readlines()
            text = ' '.join(lines)
            content.append(text)
            file.close()

dataset = {'prompt': content}

In [8]:
with open('../../data/processed/context.txt', 'w', encoding='utf8') as f:
    for c in content:
        f.write(c)
    
f.close()

In [ ]:
os.environ['OPENAI_API_KEY'] = key

In [ ]:
loader = TextLoader('../../data/processed/context.txt', encoding='utf8')

index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
print(index.query('What is the Xalaxians collective consciousness?', llm=ChatOpenAI()))